# YOLOv9 for pose estimation 

First we neede to convert all of the data we'vr created into yolo dataset.
We did it in coco_to_yolo.py

In [1]:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/student/Synthetic-data-tutorial/.conda/lib/python3.10/site-packages/traitlets/traitlets.py", line 632, in get
    value = obj._trait_values[self.name]
KeyError: '_control_lock'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/student/Synthetic-data-tutorial/.conda/lib/python3.10/site-packages/zmq/eventloop/zmqstream.py", line 575, in _log_error
    f.result()
  File "/home/student/Synthetic-data-tutorial/.conda/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 301, in dispatch_control
    async with self._control_lock:
  File "/home/student/Synthetic-data-tutorial/.conda/lib/python3.10/site-packages/traitlets/traitlets.py", line 687, in __get__
    return t.cast(G, self.get(obj, cls))  # the G should encode the Optional
  File "/home/student/Synthetic-data-tutorial/.conda/lib/python3.10/site-packa

In [2]:
# %pip install ultralytics opencv-python matplotlib pyyaml

In [1]:
from ultralytics.utils.checks import check_yolo
print(check_yolo())

Ultralytics 8.3.179 🚀 Python-3.10.18 torch-2.5.1+cu121 CUDA:0 (Tesla V100-PCIE-16GB, 16144MiB)
Setup complete ✅ (6 CPUs, 110.0 GB RAM, 122.3/145.2 GB disk)
None


In [2]:
from ultralytics import YOLO
import torch

# Load YOLOv9 Pose model (s/m/l/x, or your own .pt)
model = YOLO("yolov8s-pose.pt")

In [5]:
if torch.cuda.is_available():
    print("CUDA is available.")
    print(f"Number of CUDA devices: {torch.cuda.device_count()}")
    print(f"Current CUDA device name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")

CUDA is available.
Number of CUDA devices: 1
Current CUDA device name: Tesla V100-PCIE-16GB


close_mosaic=1

Mosaic augmentation mixes 4 images together.

Great for large human pose, but can distort small rigid objects → hurts keypoint accuracy.

close_mosaic=1 disables mosaic augmentation after the first 10 epochs (by default).

🔹 Recommended:

✅ Always set close_mosaic=1 for small rigid tools.

You can even disable it completely with mosaic=0.0 in a custom hyp.yaml.

In [6]:
model.train(
    data="surgical_tools_pose.yaml",
    imgsz=960,      # bigger img size helps small objects (If VRAM allows, go 1280 for best keypoint precision. 
                    # Avoid 640 → too coarse for tiny tools.)
    epochs=300, #200 is a good default for small datasets.
                # If dataset is very small (<5k images), you may need 300–400.
                # Use early_stopping (Ultralytics auto monitors val) to stop if performance plateaus.
    batch=32,   # Larger batch → smoother gradient, but requires more VRAM.
                # Pick the largest batch your GPU fits at chosen imgsz.
                #For 24GB VRAM: batch=32–64 at imgsz=960.
                #For 12GB VRAM: batch=16 is usually safe.
    close_mosaic=1,
    workers=8,
    patience=50  
)

Ultralytics 8.3.179 🚀 Python-3.10.18 torch-2.5.1+cu121 CUDA:0 (Tesla V100-PCIE-16GB, 16144MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=1, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=surgical_tools_pose.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-pose.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=50, perspective=0.0, plots=True, pose=12.0

Overriding model.yaml kpt_shape=[17, 3] with kpt_shape=[5, 3]
Overriding model.yaml nc=1 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]

train: Scanning /home/student/Synthetic-data/out/train/labels.cache... 1500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1500/1500 [00:00<?, ?it/s]

WARNING ⚠️ No 'flip_idx' array defined in data.yaml, disabling 'fliplr' and 'flipud' augmentations.


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 394.3±100.8 MB/s, size: 407.5 KB)


...

                   all        250        250      0.999          1      0.995      0.931      0.994      0.995      0.993      0.971



300 epochs completed in 1.874 hours.
Optimizer stripped from runs/pose/train/weights/last.pt, 23.2MB
Optimizer stripped from runs/pose/train/weights/best.pt, 23.2MB

Validating runs/pose/train/weights/best.pt...
Ultralytics 8.3.179 🚀 Python-3.10.18 torch-2.5.1+cu121 CUDA:0 (Tesla V100-PCIE-16GB, 16144MiB)
YOLOv8s-pose summary (fused): 81 layers, 11,413,731 parameters, 0 gradients, 29.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.90it/s]


                   all        250        250      0.993      0.999      0.995      0.938      0.995      0.995      0.993      0.976
         needle_holder        150        150      0.995          1      0.995      0.955      0.999          1      0.995       0.98
              tweezers        100        100       0.99      0.999      0.995      0.922       0.99       0.99      0.992      0.972
Speed: 0.3ms preprocess, 1.8ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/pose/train


ultralytics.utils.metrics.PoseMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fd2e0aac3a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(P)', 'F1-Confidence(P)', 'Precision-Confidence(P)', 'Recall-Confidence(P)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.03403

Warning about flips
WARNING No 'flip_idx' array defined in data.yaml, disabling fliplr/flipud.


If you want horizontal flip augmentation for pose, you must tell the model how your keypoints swap under a left–right flip.
Given your 5 kps are ["tl","tr","br","bl","center"], add this to surgical_tools_pose.yaml:

kpt_shape: [5, 3]
flip_idx: [1, 0, 3, 2, 4]  # tl↔tr, br↔bl, center→center


If flips don’t make sense for your tools, you can leave it disabled (current behavior).

In [5]:
model = YOLO("runs/pose/train/weights/best.pt")

results = model.predict(
    source="/datashare/project/vids_test/4_2_24_A_1_small.mp4",
    imgsz=960,
    conf=0.25,
    iou=0.5,
    device=0,        # or "cpu"
    half=True,
    save=True,
    save_txt=True,
    save_conf=True,
    vid_stride=1,
    max_det=10,
    name="eval_4_2_24_A_1"
)



WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/5395) /datashare/project/vids_test/4_2_24_A_1_small.mp4: 544x960 (no detections), 141.3ms
video 1/1 (frame 2/5395) /datashare/project/vids_test/4_2_24_A_1_small.mp4: 544x960 (no detections), 15.8ms
video 1/1 (frame 3/5395) /datashare/project/vids_test/4_2_24_A_1_small.mp4: 544x960 (no detections), 12.1ms
video 1/1 (frame 4/5395) /datashare/project/vids_test/4_2_24_A_1_small.mp4: 544x960 (no detections), 11.6ms
video 1/1 (frame 5/5395